In [1]:
import pandas as pd
import numpy as np
import os.path
import recsys as rs
import notipy
import math
import sys
import UserBasedRecommender as UBR
import random
from scipy import sparse as sps
import importlib

In [2]:
train = pd.read_csv('Data/train_final.csv','\t')
tracks = pd.read_csv('Data/tracks_final.csv','\t')
pl_info = pd.read_csv('Data/playlists_final.csv','\t')
tgt_pl = pd.read_csv('Data/target_playlists.csv','\t')
tgt_tr = pd.read_csv('Data/target_tracks.csv','\t')

In [3]:
train, test, tgt_tr, tgt_pl = rs.split_train_test(train, 10, 20, 5, 2517)

In [4]:
UBR = importlib.reload(UBR)
ubr = UBR.UserBasedRecommender(shrinkage=20,n_el_sim=20)
ubr.fit(tracks,train,tgt_tr)
print('Model fitted!')

  1%|          | 54/4420 [00:00<00:08, 509.15it/s]

Calculated Indices


  1%|          | 26/4420 [00:00<00:17, 257.98it/s]

(4420, 100000)
Model URM built
4420


100%|██████████| 4420/4420 [00:32<00:00, 135.69it/s]

Similarity built
(4420, 4420)
Model fitted!


In [5]:
_, _, ubr.IX_tgt_playlists, _ = rs.create_sparse_indexes(tracks_info=tracks, playlists=tgt_pl, tracks_reduced=tgt_tr)
URM = rs.create_tgt_URM(ubr.IX_tgt_playlists, ubr.IX_items, train)
URM = URM.tocsr()
print('URM built')

100%|██████████| 4420/4420 [00:14<00:00, 305.55it/s]


URM built


In [25]:
rs = importlib.reload(rs)
sim_check = True
secondary_sorting=True
recommendetions = np.array([])
for p in ubr.IX_tgt_playlists.values:
    ubr.S = ubr.S.tocsr()
    avg_sims = ubr.S[p,:].dot(URM).toarray().ravel()
    map = ubr.IX_items[ubr.IX_tgt_items.index.values]
    ratings = np.array(avg_sims[map])
    map = pd.Series(range(len(ubr.IX_tgt_items)), index= ubr.IX_items[ubr.IX_tgt_items.index.values].index.values ) #contains the index of tgt items in IX_items
    top = rs.top5_outside_playlist(ratings, p, train, ubr.IX_tgt_playlists, map, sim_check, secondary_sorting)
    recommendetions = np.append(recommendetions, rs.sub_format(top))
    if (p % 1000 == 0):
        print('Recommended ' + str(p) + ' users over ' + str(ubr.IX_tgt_playlists.values.shape[0]))
result= pd.DataFrame({'playlist_id' : ubr.IX_tgt_playlists.index.values, 'track_ids' : recommendetions})

Recommended 0 users over 4420
Recommended 1000 users over 4420
Recommended 2000 users over 4420
Recommended 3000 users over 4420
Recommended 4000 users over 4420


In [26]:
def evaluate(results, test, eval_metric='MAP'):
    if eval_metric == 'MAP':
        APs = results.apply(calculate_AP, axis=1, args=(test,))
        res = (APs.sum())/results.shape[0]
    return res

def calculate_AP(row, test):
    p_id = row['playlist_id']
    recs = np.fromstring(row['track_ids'], dtype=float, sep=' ')

    AP = 0
    rel_sum = 0
    n_rel_items = min(test[test['playlist_id'] == p_id].shape[0],5)
    for i in range(recs.size):
        rel = 1 if ((test['playlist_id'] == int(p_id)) & (test['track_id'] == recs[i])).any() else 0
        rel_sum += rel
        P = rel_sum/(i+1)
        AP += (P * rel)/n_rel_items

    return AP

In [27]:
map_eval = evaluate(result, test, 'MAP')
print('Evaluation completed!')
map_eval

Evaluation completed!


0.0006025641025641029